In [1]:
import msal
import requests
import callbot_creds as cd

# Replace these values with your Azure app details
CLIENT_ID = cd.Application_client_ID
TENANT_ID = cd.Directory_tenant_ID

# Microsoft Graph API endpoint
GRAPH_API_ENDPOINT = "https://graph.microsoft.com/v1.0"

# Function to get an access token using Device Code Flow
def get_access_token():
    authority = f"https://login.microsoftonline.com/{TENANT_ID}"
    app = msal.PublicClientApplication(CLIENT_ID, authority=authority)

    # Initiate device code flow
    flow = app.initiate_device_flow(scopes=["https://graph.microsoft.com/.default"])

    if "user_code" not in flow:
        raise ValueError("Failed to create device flow. Check your client ID and tenant ID.")

    print(flow["message"])  # This will show a message asking the user to visit a URL and input a code

    # Authenticate the user by asking them to visit the URL and enter the code
    result = app.acquire_token_by_device_flow(flow)

    if "access_token" in result:
        return result["access_token"]
    else:
        raise Exception(f"Could not acquire an access token: {result.get('error_description')}")

# Function to create an event in the Outlook calendar
def create_calendar_event(access_token, subject, start_time, end_time, attendee_email):
    url = f"{GRAPH_API_ENDPOINT}/me/events"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    event_data = {
        "subject": subject,
        "body": {
            "contentType": "HTML",
            "content": "This is a scheduled call with the client."
        },
        "start": {
            "dateTime": start_time,  # Format: "YYYY-MM-DDTHH:MM:SS"
            "timeZone": "Eastern Standard Time"
        },
        "end": {
            "dateTime": end_time,  # Format: "YYYY-MM-DDTHH:MM:SS"
            "timeZone": "Eastern Standard Time"
        },
        "attendees": [
            {
                "emailAddress": {
                    "address": attendee_email,
                    "name": "Client Name"
                },
                "type": "required"
            }
        ]
    }

    response = requests.post(url, headers=headers, json=event_data)

    if response.status_code == 201:
        print("Event created successfully.")
    else:
        print(f"Failed to create event. Status code: {response.status_code}")
        print(f"Response: {response.text}")

# Main execution
if __name__ == "__main__":
    try:
        # Get access token
        token = get_access_token()

        # Example event data (replace with actual data from Vapi.ai)
        subject = "Meeting with Client"
        start_time = "2024-10-25T10:00:00"
        end_time = "2024-10-25T11:00:00"
        attendee_email = "clientemail@example.com"

        # Create calendar event
        create_calendar_event(token, subject, start_time, end_time, attendee_email)

    except Exception as e:
        print(f"An error occurred: {e}")


An error occurred: Unable to get authority configuration for https://login.microsoftonline.com/your_tenant_id_here. Authority would typically be in a format of https://login.microsoftonline.com/your_tenant or https://tenant_name.ciamlogin.com or https://tenant_name.b2clogin.com/tenant.onmicrosoft.com/policy.  Also please double check your tenant name or GUID is correct.


In [6]:
#import callbot_creds as cd

import callbot_creds.py as cd

print(cd.auth_token)

ModuleNotFoundError: No module named 'callbot_creds'